# LUAD CPTAC analysis

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from msresist.figures.figure3 import plotR2YQ2Y, plotclustersIndividually, ArtificialMissingness, WinsByWeight
from msresist.figures.figure1 import pca_dfs, plotClustergram
from msresist.pre_processing import preprocessing, filter_NaNpeptides, MeanCenter, Log2T
from msresist.clustering import MassSpecClustering
from msresist.motifs import MapMotifs, FormatName
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", font_scale=1.2, color_codes=True, palette="colorblind", rc={"grid.linestyle": "dotted", "axes.linewidth": 0.6})

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_rows', 1000000)
# pd.set_option('display.max_columns', 1000)

## Import and Preprocess Data

In [4]:
# X = preprocessing(CPTAC=True, log2T=True)
X = pd.read_csv("msresist/data/MS/CPTAC/CPTAC-preprocessedMotfis.csv").iloc[:, 1:]

d = X.select_dtypes(include=['float64']).T
i = X.select_dtypes(include=['object'])

In [5]:
X.head()

,Sequence,Protein,Gene,Position,C3N.01799,C3N.01799.N,C3L.01890,C3L.01890.N,C3N.00572,C3N.00572.N,C3N.02423,C3N.02423.N,C3N.02729,C3N.02729.N,C3L.00263,...,C3N.00738,C3N.00738.N,C3N.00959,C3N.00959.N,C3N.02003,C3N.02003.N,C3N.02379.N,C3N.00175,C3N.00175.N,C3N.01823,C3N.01823.N,C3L.02549,C3L.02549.N,C3L.02365,C3L.02365.N
0,AAAAAsGGGVS,NP_002577.2,PBX2,S146-p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAAAAsQQGSA,NP_001308122.1,TBL1XR1,S119-p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.071547,-0.600298,0.172339,-1.761406,0.421872,-0.372907,-0.979491,-0.083036,-0.361962,1.275506,-0.568015,0.25423,-0.378522,0.987749,-0.189486
2,AAAAAsTQAQQ,NP_001035540.1,MINDY2,S575-p,0.012034,-0.630863,-1.159479,-0.740044,0.107228,-0.016971,-0.023379,-0.481045,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAAAGsASPRS,NP_064520.2,WRNIP1,S151-p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAAAGsGPSPP,NP_060895.1,PI4K2A,S44-p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.83783,-0.993838,0.317079,...,0.031572,-0.554645,-0.033640,-1.268642,0.266790,-0.063300,-0.647500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
X_f = filter_NaNpeptides(X, cut=0.1)
d_f = X_f.select_dtypes(include=['float64']).T
i_f = X_f.select_dtypes(include=['object'])


distance_method = "PAM250"
ncl = 20
SeqWeight = 50

MSC = MassSpecClustering(i_f, ncl, SeqWeight=SeqWeight, distance_method=distance_method, n_runs=1).fit(d_f, "NA")

Re-initialize GMM clusters, empty cluster(s) at iteration 0
Re-initialize GMM, NaN responsibilities at iteration 1
Re-initialize GMM clusters, empty cluster(s) at iteration 2
Re-initialize GMM, NaN responsibilities at iteration 3


In [ ]:
raise SystemExit

## Compare White vs Broad Quantitation

### Import White data set and Find matching values in full CPTAC data set

Import White data set:

In [ ]:
white = pd.read_csv("msresist/data/MS/CPTAC/03CPTAC_filtered_White.csv").iloc[:-1, :]

In [ ]:
white = white[~white["Protein"].str.contains("HLA class I histocompatibility antigen, B")]
white = white[~white["Protein"].str.contains("HLA class I histocompatibility antigen, A")]
white = white[~white["Protein"].str.contains("Lamina-associated polypeptide 2")]

Extract gene names from full protein description:

In [ ]:
def FormatName(X):
    """ Keep only the general protein name, without any other accession information """
    genes = []
    counter = 0
    for v in X.iloc[:, 0]:
        if "GN" not in v:
            counter += 1
            continue
        genes.append(v.split("GN=")[1].split(" PE")[0].strip())
    print("number of proteins without gene name:", counter)
    return genes

def FixWrongSequences(X):
    s1 = X[X["Sequence"].str.contains("GGSYTQAASSDSAQGSDVSLTACK")]
    

In [ ]:
white = white.set_index("Protein").drop("Uncharacterized protein FLJ45252 OS=Homo sapiens OX=9606 PE=2 SV=2").reset_index()
genes = FormatName(white)
white = white.drop("Protein", axis=1)
white.insert(1, "Gene", genes)

Process sequences to generate +/- 5 motifs:

In [ ]:
white = MapMotifs(white, white.iloc[:, 1]).drop("Position", axis=1).iloc[:, :10]

In [ ]:
white = MeanCenter(Log2T(white, list(white.columns[2:])), list(white.columns[2:]), mc_row=True, mc_col=False).sort_values(by="Sequence")

In [ ]:
white.shape

Find matching patients of CPTAC data set:

In [ ]:
WhiteCols = ["C3N.02587", "C3N.02587.N", "C3L.00893", "C3L.00893.N", "C3N.01488", "C3N.01488.N", "C3N.01413", "C3N.01413.N"]
cols = [X.columns[0]] + [X.columns[2]] + WhiteCols
X_tmt3 = X[cols].dropna().sort_values(by="Sequence")

Find matching peptides of CPTAC data set

In [ ]:
X_tmt3 = X_tmt3.set_index(["Gene", "Sequence"])
rows = []
counter = 0
for idx in range(white.shape[0]):
    try:
        r = X_tmt3.loc[white["Gene"][idx], white["Sequence"][idx]].reset_index()
        if len(r) > 1:
            rows.append(pd.DataFrame(r.iloc[0, :]).T)
        else:
            rows.append(r)
    except:
        counter += 1
        continue
print("Number of mismatches: ", counter)

In [ ]:
X_white = pd.concat(rows)
X_white = X_white.set_index(np.arange(X_white.shape[0]))

In [ ]:
X_white.shape

In [ ]:
white = white.set_index(["Gene", "Sequence"])
rows = []
counter = 0
for idx in range(X_white.shape[0]):
    try:
        r = white.loc[X_white["Gene"][idx], X_white["Sequence"][idx]].reset_index()
        if len(r) > 1:
            rows.append(pd.DataFrame(r.iloc[0, :]).T)
        else:
            rows.append(r)
    except:
        counter += 1
print(counter)

In [ ]:
White_X = pd.concat(rows)

In [ ]:
X_white = X_white.drop_duplicates(list(X_white.columns), keep="first")

In [ ]:
White_X = White_X.drop_duplicates(list(White_X.columns), keep="first")

### Heatmaps

BROAD QUANTIFICATION

In [ ]:
XW_ = X_white.set_index(["Gene"]).iloc[:, 1:]
XW_ = np.array(XW_, dtype="float64")
plotClustergram(XW_, "BI Quantification", np.max(np.abs(XW_)) * 0.3, figsize=(6,6))

WHITE QUANTIFICATION

In [ ]:
WX_ = White_X.set_index(["Gene"]).iloc[:, 1:]
WX_ = np.array(WX_, dtype="float64")
plotClustergram(WX_, "White Lab Quantification", np.max(np.abs(WX_)) * 0.6, figsize=(6,6))

### Model

#### BROAD

In [ ]:
for i in range(2, X_white.shape[1]):
    X_white.iloc[:, i] = pd.to_numeric(X_white.iloc[:, i], errors="ignore")

In [ ]:
Xd = X_white.iloc[:, 2:].T
Xi = X_white.iloc[:, :2]

In [ ]:
distance_method = "PAM250"
ncl = 9
SeqWeight = 10

In [ ]:
X_model = MassSpecClustering(Xi, ncl, SeqWeight=SeqWeight, distance_method=distance_method, n_runs=1).fit(Xd, "NA")

In [ ]:
Xcenters = X_model.transform(Xd).T

In [ ]:
plotClustergram(Xcenters, "BI clusters", max(np.max(np.abs(Xcenters))) * 0.9, figsize=(6,6))

#### Cluster Averages

In [ ]:
plotclustersIndividually(Xcenters, X_white.columns[2:], 3, 3)

#### PCA

In [ ]:
Xcenters = Xcenters.T

In [ ]:
Xcenters["Patient_ID"] = X_white.columns[2:]

In [ ]:
c=2
pp = PCA(n_components=c)
dScor_ = pp.fit_transform(Xcenters.iloc[:, :-1])
dLoad_ = pp.components_
dScor_, dLoad_ = pca_dfs(dScor_, dLoad_, Xcenters, c, ["Patient_ID"], "Cluster")

In [ ]:
dScor_.columns = ["PC1", "PC2", "Patient_ID"]

In [ ]:
varExp = np.round(pp.explained_variance_ratio_, 2)
varExp

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.scatterplot(x="PC1", y="PC2", data=dScor_, hue="Patient_ID", ax=ax, s=80, **{'linewidth':.5, 'edgecolor':"k"})
ax.set_title("PCA Scores", fontsize=15)
ax.set_xlabel("PC1 (" + str(int(varExp[0] * 100)) + "%)", fontsize=14)
ax.set_ylabel("PC2 (" + str(int(varExp[1] * 100)) + "%)", fontsize=14)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, labelspacing=0.2, fontsize=15);
# plt.savefig("PCA_CPTAC_20cl_Scores.svg", bbox_inches='tight', format='svg', dpi=1200)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.scatterplot(x="PC1", y="PC2", data=dLoad_, hue="Cluster", ax=ax, s=80, **{'linewidth':.5, 'edgecolor':"k"})
for line in range(0, dLoad_.shape[0]):
     ax.text(dLoad_.iloc[line, 0]+0.005, dLoad_.iloc[line, 1]+0.005, dLoad_["Cluster"][line], horizontalalignment='right', size='medium', color='black', weight='semibold')
ax.legend().remove()
ax.set_title("PCA Loadings", fontsize=15)
ax.set_xlabel("PC1 (" + str(int(varExp[0] * 100)) + "%)", fontsize=14)
ax.set_ylabel("PC2 (" + str(int(varExp[1] * 100)) + "%)", fontsize=14);
# plt.savefig("PCA_CPTAC_20cl_Loadings.svg", bbox_inches='tight', format='svg', dpi=1200)

#### WHITE Lab

In [ ]:
for i in range(2, White_X.shape[1]):
    White_X.iloc[:, i] = pd.to_numeric(White_X.iloc[:, i], errors="ignore")

In [ ]:
Wd = White_X.iloc[:, 2:].T
Wi = White_X.iloc[:, :2]

In [ ]:
w_model = MassSpecClustering(Wi, ncl, SeqWeight=SeqWeight, distance_method=distance_method, n_runs=1).fit(Wd, "NA")

In [ ]:
Wcenters = w_model.transform(Wd).T

In [ ]:
plotClustergram(Wcenters, "White", max(np.max(np.abs(Wcenters))) * 0.9, figsize=(6,6))

#### Cluster Averages

In [ ]:
plotclustersIndividually(Wcenters, X_white.columns[2:], 3, 3)

#### PCA

In [ ]:
Wcenters = Wcenters.T

In [ ]:
Wcenters["Patient_ID"] = X_white.columns[2:]

In [ ]:
c=2
pp = PCA(n_components=c)
dScor_ = pp.fit_transform(Wcenters.iloc[:, :-1])
dLoad_ = pp.components_
dScor_, dLoad_ = pca_dfs(dScor_, dLoad_, Wcenters, c, ["Patient_ID"], "Cluster")

In [ ]:
dScor_.columns = ["PC1", "PC2", "Patient_ID"]

In [ ]:
varExp = np.round(pp.explained_variance_ratio_, 2)
varExp

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.scatterplot(x="PC1", y="PC2", data=dScor_, hue="Patient_ID", ax=ax, s=80, **{'linewidth':.5, 'edgecolor':"k"})
ax.set_title("PCA Scores", fontsize=15)
ax.set_xlabel("PC1 (" + str(int(varExp[0] * 100)) + "%)", fontsize=14)
ax.set_ylabel("PC2 (" + str(int(varExp[1] * 100)) + "%)", fontsize=14)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, labelspacing=0.2, fontsize=15);
# plt.savefig("PCA_CPTAC_20cl_Scores.svg", bbox_inches='tight', format='svg', dpi=1200)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.scatterplot(x="PC1", y="PC2", data=dLoad_, hue="Cluster", ax=ax, s=80, **{'linewidth':.5, 'edgecolor':"k"})
for line in range(0, dLoad_.shape[0]):
     ax.text(dLoad_.iloc[line, 0]+0.005, dLoad_.iloc[line, 1]+0.005, dLoad_["Cluster"][line], horizontalalignment='right', size='medium', color='black', weight='semibold')
ax.legend().remove()
ax.set_title("PCA Loadings", fontsize=15)
ax.set_xlabel("PC1 (" + str(int(varExp[0] * 100)) + "%)", fontsize=14)
ax.set_ylabel("PC2 (" + str(int(varExp[1] * 100)) + "%)", fontsize=14);
# plt.savefig("PCA_CPTAC_20cl_Loadings.svg", bbox_inches='tight', format='svg', dpi=1200)

## Distribution of Missingness

In [ ]:
p_nan_counts = []
for i in range(d.shape[1]):
    p_nan_counts.append(np.count_nonzero(np.isnan(d[i])))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
ax = sns.distplot(p_nan_counts, 10)
ax.set_title("Missingness distribution in LUAD")
ax.set_ylabel("Density")
ax.set_xlabel("Number of missing observations per peptide")

print("distribution mean:", np.mean(p_nan_counts))

## Artificial Missingness

Hypothesis: The sequence motif helps to correctly assign peptides to representative clusters, particularly at higher levels of missingness.

Approach: Introduce artificial missing data by leaving values out, and then seeing how well that values are represented by the corresponding clusters.

In [ ]:
cd = filter_NaNpeptides(X, cut=1)
assert True not in np.isnan(cd.iloc[:, 4:]), "There are still NaNs."
cd.head()

Generate data sets with 5%, 10%, 20%, 40%, 60%, 80%, 90% missingness:

In [ ]:
%%time
nan_per = [0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 0.9]
weights = [0, 0.25, 1000]
ncl = 5

am = ArtificialMissingness(cd, weights, nan_per, "PAM250", ncl)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 6))
sns.lineplot(x="Missing%", y="Error", data=am, hue="Prioritize")
# plt.savefig("Error_ArtificialMissingness.svg", bbox_inches='tight', format='svg', dpi=1200)

In [ ]:
# %%time
# model = MassSpecClustering(i_w, ncl, SeqWeight=500, distance_method="PAM250", n_runs=1).fit(d_w, "NA")
# model.wins_

## Weight Variation

Plot sequence, data, both, and mix wins with different SequenceWeights

In [ ]:
X_w = filter_NaNpeptides(X, cut=0.5)
d_w = X_w.select_dtypes(include=['float64']).T
i_w = X_w.select_dtypes(include=['object'])

In [ ]:
%%time
weights = np.arange(0, 110, 10)
ww = WinsByWeight(i_w, d_w, weights, "PAM250")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 6))
sns.lineplot(x="Sequence_Weighting", y="Wins", data=ww, hue="Prioritize")
plt.savefig("WinsAcrossWeigths.svg", bbox_inches='tight', format='svg', dpi=1200)

In [ ]:
raise SystemExit

## Run Model

### Filter peptides based on a threshold of missingness

In [ ]:
X_f = filter_NaNpeptides(X, cut=0.1)
d_f = X_f.select_dtypes(include=['float64']).T
i_f = X_f.select_dtypes(include=['object'])

Reduce data set:

In [ ]:
# Reduce data set?
# npept = 7000
# d = d.iloc[:, :npept]
# i = i.iloc[:npept, :]

#Drop NaN?
# d = d.iloc[:8, :].dropna(axis=1)
# i = i.iloc[d.columns, :]

Run model:

In [ ]:
# %%time
# distance_method = "PAM250"
# ncl = 20
# SeqWeight = 50

# MSC = MassSpecClustering(i_f, ncl, SeqWeight=SeqWeight, distance_method=distance_method, n_runs=1).fit(d_f, "NA")

Save model:

In [ ]:
# with open('CPTACmodel', 'wb') as f:
#     pickle.dump([MSC], f)

Load if necessary:

In [ ]:
# with open('CPTACmodel_w50S_c20', 'rb') as f:
#     MSC = pickle.load(f)[0]

## Hierarchical Clustering

In [ ]:
centers = MSC.transform(d_f)
centers["Patient_ID"] = X.columns[4:]

In [ ]:
MSC.wins_

In [ ]:
centers = centers.set_index(["Patient_ID"])
plotClustergram(centers, "", lim=max(np.max(np.abs(centers)) * 0.5), figsize=(10, 10), yticklabels=True)
centers = centers.reset_index()
# plt.savefig("CPTAC_GMM.svg", bbox_inches='tight', format='svg', dpi=1200)

## Regression against Clinical Features

Import 2y-Survival data:

In [ ]:
cf = pd.read_csv("msresist/data/MS/CPTAC/Y_CPTAC.csv").drop("Metastasis", axis=1)
cf_ = cf.replace("Living", 0)
cf_ = cf_.replace("Deceased", 1)
cf_ = cf_.replace("No", 0)
cf_ = cf_.replace("Yes", 1)

In [ ]:
cf_.shape

Import patient accessions to match signaling with clinical data:

In [ ]:
IDict = pd.read_csv("msresist/data/MS/CPTAC/IDs.csv", header=0)
IDict_ = dict(zip(IDict.iloc[:, 0], IDict.iloc[:, 1]))

Change patient IDs from Case ID to Broad ID:

In [ ]:
ids = []
for i in range(cf.shape[0]):
    ids.append(IDict_[cf_.iloc[i, 0]])
cf_["Patient_ID"] = ids

Add Tumor row per patient:

In [ ]:
for i in range(cf_.shape[0]):
    id_ = cf_.iloc[i, 0].split(".N")[0]
    cf_.loc[-1] = [id_, cf_.iloc[i, 1]]
    cf_.index = cf_.index + 1
cf_.loc[-1] = ["C3N.02379.1", 0]
cf_.index = cf_.index + 1
cf_ = cf_.sort_index()

In [ ]:
centers.head()

In [ ]:
centers = centers.set_index("Patient_ID").drop(["C3N.02587", "C3N.02587.N"], axis=0).sort_values(by="Patient_ID")
y = pd.read_csv("msresist/data/MS/CPTAC/y.csv").sort_values(by="Patient_ID").set_index("Patient_ID")

### R2Y/Q2Y

In [ ]:
p1 = PLSRegression(n_components=2, scale=False)
fig, ax = plt.subplots(1, 1, figsize=(7,6))
plotR2YQ2Y(ax, p1, centers, y, 1, 10)

In [ ]:
centers = centers.reset_index()

## PCA Analysis

In [ ]:
c=2
pp = PCA(n_components=c)
dScor_ = pp.fit_transform(centers.iloc[:, 1:])
dLoad_ = pp.components_
dScor_, dLoad_ = pca_dfs(dScor_, dLoad_, centers, c, ["Patient_ID"], "Cluster")

In [ ]:
dScor_.columns = ["PC1", "PC2", "Patient_ID"]

In [ ]:
tumortype = []
for i in range(dScor_.shape[0]):
    if ".N" in dScor_["Patient_ID"][i]:
        tumortype.append("Normal")
    else:
        tumortype.append("Tumor")

In [ ]:
dScor_["Type"] = tumortype

In [ ]:
varExp = np.round(pp.explained_variance_ratio_, 2)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
sns.scatterplot(x="PC1", y="PC2", data=dScor_, hue="Type", style="Type", ax=ax, s=80, **{'linewidth':.5, 'edgecolor':"k"})
ax.set_title("PCA Scores", fontsize=15)
ax.set_xlabel("PC1 (" + str(int(varExp[0] * 100)) + "%)", fontsize=14)
ax.set_ylabel("PC2 (" + str(int(varExp[1] * 100)) + "%)", fontsize=14)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, labelspacing=0.2, fontsize=15);
# plt.savefig("PCA_CPTAC_20cl_Scores.svg", bbox_inches='tight', format='svg', dpi=1200)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
sns.scatterplot(x="PC1", y="PC2", data=dLoad_, hue="Cluster", ax=ax, s=80, **{'linewidth':.5, 'edgecolor':"k"})
for line in range(0, dLoad_.shape[0]):
     ax.text(dLoad_.iloc[line, 0]+0.001, dLoad_.iloc[line, 1]+0.009, dLoad_["Cluster"][line], horizontalalignment='right', size='medium', color='black', weight='semibold')
ax.legend().remove()
ax.set_title("PCA Loadings", fontsize=15)
ax.set_xlabel("PC1 (" + str(int(varExp[0] * 100)) + "%)", fontsize=14)
ax.set_ylabel("PC2 (" + str(int(varExp[1] * 100)) + "%)", fontsize=14);
# plt.savefig("PCA_CPTAC_20cl_Loadings.svg", bbox_inches='tight', format='svg', dpi=1200)

In [ ]:
clustermembers = MSC.clustermembers(X_f.T)
clustermembers.head(5)